In [ ]:
#Import libreries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

In [ ]:
# STEP 1: Create a dataframe to work on it and see a preview of the data
from google.colab import drive
drive.mount('/content/drive')
path = '/content/drive/MyDrive/online_retail_II.csv'
df = pd.read_csv(path)
df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12.0,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12.0,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12.0,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48.0,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24.0,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [ ]:
df.tail()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
850424,565426,44234,ASSORTED CIRCULAR MOBILE,1.0,2011-09-04 12:41:00,0.21,12851.0,United Kingdom
850425,565426,44234,ASSORTED CIRCULAR MOBILE,1.0,2011-09-04 12:41:00,0.21,12851.0,United Kingdom
850426,565426,44234,ASSORTED CIRCULAR MOBILE,1.0,2011-09-04 12:41:00,0.21,12851.0,United Kingdom
850427,565426,22666,RECIPE BOX PANTRY YELLOW DESIGN,1.0,2011-09-04 12:41:00,2.95,12851.0,United Kingdom
850428,565426,23251,VINTAGE RED EN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# STEP 2: Now we need to see the sum of nulls and verify if we can save that data
df.isnull().sum()

,0
Invoice,0
StockCode,0
Description,4098
Quantity,1
InvoiceDate,1
Price,1
Customer ID,197816
Country,1


In [ ]:
# This helps us understand what kind of transactions have no user attached
rows_with_null_customer = df[df['Customer ID'].isnull()]

print(f"Count of missing Customers: {rows_with_null_customer.shape[0]}")
display(rows_with_null_customer.head(10))

Count of missing Customers: 197816


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
263,489464,21733,85123a mixed,-96.0,2009-12-01 10:52:00,0.00,NaN,United Kingdom
283,489463,71477,short,-240.0,2009-12-01 10:52:00,0.00,NaN,United Kingdom
284,489467,85123A,21733 mixed,-192.0,2009-12-01 10:53:00,0.00,NaN,United Kingdom
470,489521,21646,NaN,-50.0,2009-12-01 11:44:00,0.00,NaN,United Kingdom
577,489525,85226C,BLUE PULL BACK RACING CAR,1.0,2009-12-01 11:49:00,0.55,NaN,United Kingdom
578,489525,85227,SET/6 3D KIT CARDS FOR KIDS,1.0,2009-12-01 11:49:00,0.85,NaN,United Kingdom
1055,489548,22271,FELTCRAFT DOLL ROSIE,1.0,2009-12-01 12:32:00,2.95,NaN,United Kingdom
1056,489548,22254,FELT TOADSTOOL LARGE,12.0,2009-12-01 12:32:00,1.25,NaN,United Kingdom
1057,489548,22273,FELTCRAFT DOLL MOLLY,3.0,2009-12-01 12:32:00,2.95,NaN,United Kingdom
1058,489548,22195,LARGE HEART MEASURING SPOONS,1.0,2009-12-01 12:32:00,1.65,NaN,United Kingdom


In [ ]:
# The same with "Description" usually these are system adjustments or errors
rows_with_null_description = df[df['Description'].isnull()]

print(f"Count of missing Descriptions: {rows_with_null_description.shape[0]}")
display(rows_with_null_description.head(10))

Count of missing Descriptions: 4098


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
470,489521,21646,NaN,-50.0,2009-12-01 11:44:00,0.0,NaN,United Kingdom
3114,489655,20683,NaN,-44.0,2009-12-01 17:26:00,0.0,NaN,United Kingdom
3161,489659,21350,NaN,230.0,2009-12-01 17:39:00,0.0,NaN,United Kingdom
3731,489781,84292,NaN,17.0,2009-12-02 11:45:00,0.0,NaN,United Kingdom
4296,489806,18010,NaN,-770.0,2009-12-02 12:42:00,0.0,NaN,United Kingdom
4566,489821,85049G,NaN,-240.0,2009-12-02 13:25:00,0.0,NaN,United Kingdom
6378,489882,35751C,NaN,12.0,2009-12-02 16:22:00,0.0,NaN,United Kingdom
6555,489898,79323G,NaN,954.0,2009-12-03 09:40:00,0.0,NaN,United Kingdom
6576,489901,21098,NaN,-200.0,2009-12-03 09:47:00,0.0,NaN,United Kingdom
6581,489903,21166,NaN,48.0,2009-12-03 09:57:00,0.0,NaN,United Kingdom


In [ ]:
#  STEP 3: Standardize Column Names

df.rename(columns={
    'Invoice': 'InvoiceNo',
    'Price': 'UnitPrice',
    'Customer ID': 'CustomerID'
}, inplace=True)

#  STEP 4: Handle Missing Values
# Drop rows where CustomerID is null because we cannot track retention for unknown users

df_clean = df.dropna(subset=['CustomerID'])

# STEP 5: Filter Invalid Transactions

df_clean = df_clean[(df_clean['Quantity'] > 0) & (df_clean['UnitPrice'] > 0)]

#  STEP 6: Create column "Total price"

df_clean['TotalAmount'] = df_clean['Quantity'] * df_clean['UnitPrice']

#  STEP 7: Fix Data Types

df_clean['CustomerID'] = df_clean['CustomerID'].astype(int)

#  STEP 8: Final Check
print(f"Original Row Count: {df.shape[0]}")
print(f"Clean Row Count: {df_clean.shape[0]}")
df_clean.head()

Original Row Count: 850429
Clean Row Count: 637117


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalAmount
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12.0,2009-12-01 07:45:00,6.95,13085,United Kingdom,83.4
1,489434,79323P,PINK CHERRY LIGHTS,12.0,2009-12-01 07:45:00,6.75,13085,United Kingdom,81.0
2,489434,79323W,WHITE CHERRY LIGHTS,12.0,2009-12-01 07:45:00,6.75,13085,United Kingdom,81.0
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48.0,2009-12-01 07:45:00,2.10,13085,United Kingdom,100.8
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24.0,2009-12-01 07:45:00,1.25,13085,United Kingdom,30.0


In [ ]:

# STEP 9: Calculate RFM Metrics

# Convert 'InvoiceDate' to datetime objects
df_clean['InvoiceDate'] = pd.to_datetime(df_clean['InvoiceDate'])

# We need a reference date to calculate "Recency".

# Usually, we take the max date in the dataset + 1 day to ensure recency is at least 1.

ref_date = df_clean['InvoiceDate'].max() + dt.timedelta(days=1)

# Group by CustomerID and calculate the 3 metrics

# Recency: Days since last purchase
# Frequency: Count of unique orders
# Monetary: Total money spent

rfm = df_clean.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (ref_date - x.max()).days,
    'InvoiceNo': 'nunique',
    'TotalAmount': 'sum'
}).reset_index()

# Rename columns to standard RFM terms
rfm.rename(columns={
    'InvoiceDate': 'Recency',
    'InvoiceNo': 'Frequency',
    'TotalAmount': 'Monetary'
}, inplace=True)

# STEP 10: Calculate RFM Scores (1-5)

# We use pd.qcut to divide data into 5 equal parts (quantiles)

# Recency: LOWER is better (bought recently = 5, bought long ago = 1)
rfm['R_Score'] = pd.qcut(rfm['Recency'], 5, labels=[5, 4, 3, 2, 1])

# Frequency: HIGHER is better.

# We use .rank(method='first') to handle ties (many customers have same frequency)

rfm['F_Score'] = pd.qcut(rfm['Frequency'].rank(method='first'), 5, labels=[1, 2, 3, 4, 5])

# Monetary: HIGHER is better

rfm['M_Score'] = pd.qcut(rfm['Monetary'], 5, labels=[1, 2, 3, 4, 5])

# Combine R and F scores into a string to help segmentation (e.g., "55", "14")

rfm['RF_Score'] = rfm['R_Score'].astype(str) + rfm['F_Score'].astype(str)

# STEP 11: Create Human-Readable Segments

# This function maps the scores to marketing names using a Regex map (standard Hibernating vs Champions logic)

def segment_customer(row):
    r = int(row['R_Score'])
    f = int(row['F_Score'])

    if r >= 4 and f >= 4:
        return 'Champions' # Bought recently and often
    elif r >= 3 and f >= 3:
        return 'Loyal Customers' # Good buys, fairly frequent
    elif r >= 4 and f <= 2:
        return 'New Customers' # Bought recently, but not often yet
    elif r <= 2 and f >= 4:
        return 'At Risk' # Used to buy a lot, but haven't seen them lately
    elif r <= 2 and f <= 2:
        return 'Hibernating' # Haven't bought in a long time and didn't buy much
    else:
        return 'Potential Loyalist' # The middle ground

# Apply the function
rfm['Segment'] = rfm.apply(segment_customer, axis=1)

# STEP 12: Final Export for Power BI
# We save this final table to build the dashboard
print(rfm[['CustomerID', 'Recency', 'Frequency', 'Monetary', 'Segment']].head())

   CustomerID  Recency  Frequency  Monetary             Segment
0       12346      230         12  77556.46     Loyal Customers
1       12347       34          6   4114.18           Champions
2       12348      153          4   1709.40     Loyal Customers
3       12349      312          3   2671.14  Potential Loyalist
4       12350      214          1    334.40  Potential Loyalist


In [ ]:
# FIX: Rounding Monetary values

# (example: 18.750000001 -> 18.75)
rfm['Monetary'] = rfm['Monetary'].round(2)

# Verify again before exporting
print(rfm.head())

   CustomerID  Recency  Frequency  Monetary R_Score F_Score M_Score RF_Score  \
0       12346      230         12  77556.46       3       5       5       35   
1       12347       34          6   4114.18       5       4       5       54   
2       12348      153          4   1709.40       3       3       4       33   
3       12349      312          3   2671.14       2       3       5       23   
4       12350      214          1    334.40       3       1       2       31   

              Segment  
0     Loyal Customers  
1           Champions  
2     Loyal Customers  
3  Potential Loyalist  
4  Potential Loyalist  


In [ ]:
# Export again
rfm.to_csv('rfm_segmentation_final.csv', index=False)